In [31]:
## pip install searchtweets
from searchtweets import ResultStream, gen_rule_payload, load_credentials

In [32]:
premium_search_args = load_credentials(".twitter_keys.yaml",
                                       yaml_key="search_tweets_api",
                                       env_overwrite=False)

Grabbing bearer token from OAUTH


In [33]:
rule = gen_rule_payload("beyonce", results_per_call=10) # testing with a sandbox account
print(rule)

{"query": "beyonce", "maxResults": 10}


In [34]:
rs = ResultStream(rule_payload=rule,
                  max_results=10,
                  max_pages=1,
                  **premium_search_args)

print(rs)

ResultStream: 
	{
    "username": null,
    "endpoint": "https://api.twitter.com/1.1/tweets/search/fullarchive/BTG.json",
    "rule_payload": {
        "query": "beyonce",
        "maxResults": 10
    },
    "tweetify": true,
    "max_results": 10
}


In [41]:
## makes 1 request every time you call this (limit 50 requests/month)
tweets = list(rs.stream())

In [42]:
[print(tweet.all_text) for tweet in tweets[0:10]];